<a href="https://colab.research.google.com/github/anandhulk/MainProject/blob/main/Model_Evaluation_ROUGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 10.2 MB/s eta 0:00:00


In [15]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

In [3]:
from datasets import load_from_disk

# Load a dataset
dataset = load_from_disk('/content/drive/MyDrive/test_dataset')

In [5]:
dataset

Dataset({
    features: ['article', 'summary', 'section_headings', 'keywords', 'year', 'title'],
    num_rows: 500
})

In [ ]:
model = PegasusForConditionalGeneration.from_pretrained('pegasus')

In [ ]:
tokenizer = PegasusTokenizer.from_pretrained('pegasus')

In [ ]:
!pip install tqdm


[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from tqdm import tqdm

summaries = []

for example in tqdm(dataset):
    input_ids = tokenizer.encode(example['article'],max_length=1024,truncation=True, return_tensors='pt')
    summary_ids = model.generate(input_ids)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    summaries.append(summary)

  0%|                                                                                          | 0/500 [00:00<?, ?it/s]C:\Users\user\miniconda3\envs\myenv\lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 128 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|█████████████████████████████████████████████████████████████████████████████| 500/500 [13:33:05<00:00, 97.57s/it]


In [8]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=7e249d0657440729f8dbae8b1cb9e4005611b052ee4791c15d01eb8d257205dd
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge_score


In [9]:
from datasets import load_metric
rouge = load_metric("rouge")

In [14]:
# Calculate the ROUGE score
references = [example['summary'] for example in dataset]
scores = rouge.compute(predictions=summaries, references=references)
print(scores)

{'rouge1': AggregateScore(low=Score(precision=0.5374215952832115, recall=0.27275387270643037, fmeasure=0.35808391683287744), mid=Score(precision=0.5474094597001713, recall=0.2791789183256743, fmeasure=0.36484342317643864), high=Score(precision=0.5583178535105614, recall=0.2853011024043369, fmeasure=0.37159648713110094)), 'rouge2': AggregateScore(low=Score(precision=0.18414488125795322, recall=0.09264681503346649, fmeasure=0.12179916165891745), mid=Score(precision=0.19554827771475572, recall=0.09835625519198862, fmeasure=0.1290720860778377), high=Score(precision=0.20637560456466578, recall=0.10416771845000355, fmeasure=0.13642719348573148)), 'rougeL': AggregateScore(low=Score(precision=0.31365051534362326, recall=0.15884488297921304, fmeasure=0.20866191949476223), mid=Score(precision=0.32329896252865425, recall=0.16422527414097077, fmeasure=0.21491577674246215), high=Score(precision=0.33389840014988076, recall=0.16993388970739187, fmeasure=0.22160018406168752)), 'rougeLsum': AggregateSc

In [16]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, scores[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = ['pegasus'])

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.364843,0.129072,0.214916,0.214935
